In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
raw_data_dir = Path.cwd() / "data" / "raw"
clean_data_dir = Path.cwd() / "data" / "clean"

# Taylor Rule

In [3]:
time_span = [year for year in range(1980, 2003)]

### Interest Rates

(OECD)

In [19]:
df = pd.read_csv(
    raw_data_dir / "ST_interestrate_1980-2002.csv",
    sep=";",  
)

In [20]:
df = df.rename(columns={"Year": "year", "Interest Rate": "interest_rate"})

In [21]:
df = df.loc[df["year"].isin(time_span)]

In [22]:
df["interest_rate"] = df["interest_rate"].apply(lambda x: x.replace(",", ".")).astype(float)

In [24]:
df.head()

,year,interest_rate
0,1980,16.79
1,1981,19.23
2,1982,19.91
3,1983,18.31
4,1984,17.27


In [25]:
df.to_csv(clean_data_dir / "interest_rate.csv", sep="\t", index=False)

### Inflation

(WWW.RIVALUTA.it)

In [26]:
df = pd.read_csv(
    raw_data_dir / "datiinflazionemediaitalia.csv", 
    sep=";",  
)

In [27]:
df = df[1:-1]

In [28]:
df = df.rename(columns={"WWW.RIVALUTA.it": "inflation_rate"})

In [29]:
df.reset_index(col_level=0, names="year", inplace=True)

In [30]:
df = df.sort_values("year")

In [31]:
df["year"] = df["year"].astype(int)

In [32]:
df = df.loc[df["year"].isin(time_span)]

In [33]:
df["inflation_rate"] = df["inflation_rate"].apply(
    lambda x: float(x.replace(",", ".").strip("%")) / 100.0
)

In [34]:
df.head()

,year,inflation_rate
43,1980,0.212
42,1981,0.178
41,1982,0.165
40,1983,0.147
39,1984,0.108


In [35]:
df.to_csv(clean_data_dir / "inflation_rate.csv", sep="\t", index=False)

### Inflation Target

(approximation reading papers)

The idea of targeting inflation has developed in the early 2000's once the ECB was introduced. 
Therefore, no clear-cut data was found. However, we know that the Italy was aiming at an inflation of about 5-7 % in the early 80's which started decerasing during the course of the years down to the iconic 2 %. 

Down here is the data estimated by Mario Sarcinelli.

In [36]:
years = list(range(1980, 2003))

inflation_target = np.array([12, 11.5, 10.5, 10, 10, 7, 6, 4, 4.5, 4, 4.5, 5, 4.5, 4.5, 3.5, 4.7, 4., 3, 2, 2, 2, 2, 2])
inflation_target *= 0.01

df = pd.DataFrame({'year': years, 'inflation_target': inflation_target})

In [37]:
df.head()

,year,inflation_target
0,1980,0.120
1,1981,0.115
2,1982,0.105
3,1983,0.100
4,1984,0.100


In [38]:
df.to_csv(clean_data_dir / "inflation_target.csv", sep="\t", index=False)

### Gdp

(Macrotrends)

In [39]:
df = pd.read_csv(
    raw_data_dir / "italy-gdp-gross-domestic-product.csv",
    sep=",",
    usecols=["date", "GDP ( Billions of US $)"],
)

In [40]:
df = df.rename(columns={"GDP ( Billions of US $)": "gdp"})

In [41]:
def date_to_year(date):
    date_object = datetime.strptime(date, "%Y-%m-%d")
    year = date_object.year
    return year

In [42]:
df["year"] = df["date"].apply(lambda x: date_to_year(x))
df.drop("date", axis=1, inplace=True)

In [43]:
df = df[["year", "gdp"]]

In [44]:
df = df.loc[df["year"].isin(time_span)]

In [45]:
df.head()

,year,gdp
20,1980,477.256776
21,1981,430.702851
22,1982,427.272646
23,1983,443.042374
24,1984,437.887689


In [46]:
df.to_csv(clean_data_dir / "gdp.csv", sep="\t", index=False)

### Output Gap 

(OECD, Nasdaq)

In [47]:
# work on oecd
df_oecd = pd.read_csv(
    raw_data_dir / "EO108_INTERNET_27032023112456648.csv",
    sep=",",
    usecols=["Value", "Time"]
)

# fill gaps with nasdaq
df_nasdaq = pd.read_csv(
    raw_data_dir / "nasdaq_output_gap.csv",
    sep=",",
)

In [48]:
df_oecd = df_oecd.rename(columns={"Time": "year", "Value": "output_gap"})

In [49]:
df_oecd.head()

,year,output_gap
0,1985,-2.239437
1,1986,-1.493276
2,1987,-0.898263
3,1988,0.883434
4,1989,1.800495


In [55]:
df_oecd = df_oecd.loc[df_oecd["year"].isin(time_span)]

In [56]:
df_oecd["year"].unique()
# need from 1980 to 1984

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002])

In [57]:
df_nasdaq = df_nasdaq.rename(columns={"Value": "output_gap"})

In [58]:
df_nasdaq["year"] = df_nasdaq["Date"].apply(lambda x: date_to_year(x))
df_nasdaq.drop("Date", axis=1, inplace=True)

In [59]:
df_nasdaq = df_nasdaq[["year", "output_gap"]]

In [60]:
df_nasdaq = df_nasdaq.sort_values("year")

In [61]:
df_nasdaq = df_nasdaq.loc[df_nasdaq["year"].isin([year for year in range(1980, 1985)])]

In [62]:
df = pd.concat([df_nasdaq, df_oecd])

In [63]:
df.head()

,year,output_gap
44,1980,5.396
43,1981,-0.922
42,1982,-2.350
41,1983,-3.552
40,1984,-2.860


In [64]:
df.to_csv(clean_data_dir / "output_gap.csv", sep="\t", index=False)

# Additional Regressors

### US GDP

(Macroeconomics trends)

In [65]:
df = pd.read_csv(
    raw_data_dir / "united-states-gdp-gross-domestic-product.csv",
    sep=",",
    usecols=["date", " GDP ( Billions of US $)"]
)

In [66]:
df["year"] = df["date"].apply(lambda x: date_to_year(x))
df.drop("date", axis=1, inplace=True)

In [67]:
df = df.rename(columns={" GDP ( Billions of US $)": "us_gdp"})

In [68]:
df = df[["year", "us_gdp"]]

In [69]:
df = df.loc[df["year"].isin(time_span)]

In [70]:
df = df.reset_index(drop=True)

In [71]:
df.head()

,year,us_gdp
0,1980,2857.307
1,1981,3207.041
2,1982,3343.789
3,1983,3634.038
4,1984,4037.613


In [72]:
df.to_csv(clean_data_dir / "us_gdp.csv", sep="\t", index=False)

### Unemployment

(FRED)

In [91]:
df = pd.read_csv(
    raw_data_dir / "LRHUTTTTITA156S.csv",
    sep=",",
)

In [92]:
df = df.rename(columns={"DATE": "year", "LRHUTTTTITA156S": "unemployment_rate"})

In [93]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [94]:
df = df.loc[df["year"].isin(time_span)]

In [95]:
df.head()

,year,unemployment_rate
0,1980,6.966667
1,1981,7.066667
2,1982,7.166667
3,1983,7.366667
4,1984,7.833333


In [96]:
df.to_csv(clean_data_dir / "unemployment_rate.csv", sep="\t", index=False)

### Exchange Rate

In [97]:
df = pd.read_csv(
    raw_data_dir / "USD_serie_storica_giornaliera .csv",
    sep=",",
    usecols=["Data di Riferimento", "Quotazione"]
)

In [98]:
df = df.rename(columns={"Data di Riferimento": "year", "Quotazione": "exchange_rate"})

In [99]:
df = df.sort_values("year")

In [100]:
df["year"] = df["year"].apply(lambda x: date_to_year(x))

In [101]:
df = df.drop_duplicates("year", keep='last')

In [102]:
df = df.loc[df["year"].isin(time_span)]

In [103]:
add = pd.DataFrame(
[[2002, 1950.000]],
    columns=["year", "exchange_rate"]
)

In [104]:
df = pd.concat([df, add])

In [105]:
df = df.reset_index(drop=True)

In [106]:
df.head()

,year,exchange_rate
0,1980,930.500
1,1981,1200.000
2,1982,1370.000
3,1983,1659.500
4,1984,1935.875


In [107]:
df.to_csv(clean_data_dir / "exchange_rate.csv", sep="\t", index=False)

### Foreign Interest Rate

In [108]:
df = pd.read_csv(
    raw_data_dir / "US10y_bond.csv",
    sep=",",
)

In [109]:
df = df.rename(columns={"YEAR": "year", "Value": "US_interest_rate"})

In [110]:
df = df.loc[df["year"].isin(time_span)]

In [111]:
df.head()

,year,US_interest_rate
20,1980,11.460000
21,1981,13.910833
22,1982,13.001667
23,1983,11.105000
24,1984,12.438333


In [112]:
df.to_csv(clean_data_dir / "us_bond_yield.csv", sep="\t", index=False)

### Terms of trade

(OECD)

In [121]:
df = pd.read_csv(
    raw_data_dir / "DP_LIVE_27032023194857360.csv",
    sep=",",
    usecols=["TIME", "Value"]
)

In [123]:
df = df.rename(columns={"TIME":"year", "Value":"terms_of_trade"})

In [124]:
df.head()

,year,terms_of_trade
0,1980,86.539464
1,1981,83.202130
2,1982,86.643307
3,1983,87.829031
4,1984,88.061969


In [125]:
df.to_csv(clean_data_dir / "terms_of_trade.csv", sep="\t", index=False)